In [67]:
##### 데이터 다운로드
# 한 번 함
'''
!pip install datasets   # Package install

from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드
'''

'\n!pip install datasets   # Package install\n\nfrom datasets import load_dataset   # Huggingface 데이터셋 패키지 import\ndata = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드\n'

In [68]:
# 한 번 수행함
'''
# connect google drive
from google.colab import drive
drive.mount('/content/drive')
# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh
'''

"\n# connect google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n# Download konlpy\n%cd ./drive/MyDrive/Colab\\ Notebooks/\n! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git\n%cd ./Mecab-ko-for-Google-Colab\n!bash install_mecab-ko_on_colab_light_220429.sh\n"

In [69]:
from konlpy.tag import Mecab
mecab = Mecab()

sen = "실용자연어처리 실습 진행중입니다."
print(mecab.morphs(sen))
print(mecab.nouns(sen))
print(mecab.pos(sen))

['실용', '자연어', '처리', '실습', '진행', '중', '입니다', '.']
['실용', '자연어', '처리', '실습', '진행', '중']
[('실용', 'NNG'), ('자연어', 'NNG'), ('처리', 'NNG'), ('실습', 'NNG'), ('진행', 'NNG'), ('중', 'NNB'), ('입니다', 'VCP+EF'), ('.', 'SF')]


In [70]:
##### 데이터 구조 훑어보기
# print("Data type: ", type(data))    # 데이터 타입 확인
# print("Data structure: ", data)     # 데이터 구조 확인
# print("Data keys: ", data.keys())   # 데이터 키 확인

print(data['train'][0])   # 실제 데이터 확인

{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


In [71]:
##### 테스트 세트 만들기
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

print(train_data)
print(dev_data)
print(test_data)

train_data = train_data.map(lambda example:{"label": example["label"], "text": " ".join(mecab.morphs(example["text"]))})
dev_data = dev_data.map(lambda example:{"label": example["label"], "text": " ".join(mecab.morphs(example["text"]))})
test_data = test_data.map(lambda example:{"label": example["label"], "text": " ".join(mecab.morphs(example["text"]))})

print(train_data[0])

Dataset({
    features: ['label', 'text'],
    num_rows: 36000
})
Dataset({
    features: ['label', 'text'],
    num_rows: 1333
})
Dataset({
    features: ['label', 'text'],
    num_rows: 2667
})
{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}


In [72]:
##### 텍스트 데이터 특성
# import matplotlib.pyplot as plt
# plt.hist(data['train']['label'], color='red')
# plt.show()
# plt.hist(data['validation']['label'], color='blue')
# plt.show()
# plt.hist(data['test']['label'], color='green')
# plt.show()

In [73]:
##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 하는 법 구현)
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer       # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

vectorizer = TfidfVectorizer(strip_accents="unicode", token_pattern = r"(?u)\b\w\w+\b|'\w+",ngram_range = (1,2))      # 데이터를 벡터화 해주는 모델 
vectorizer.fit(train_data['text'])  # 텍스트 문서 모음을 토큰 수의 행렬로 변환

print(vectorizer.vocabulary_)       # 텍스트 문서에 나타난 어휘의 집합을 출력
print(len(vectorizer.vocabulary_))  # 텍스트 문서에 나타난 어휘 집합의 길이를 출력

train_vectors = vectorizer.transform(train_data['text'])    # 학습 데이터를 숫자로 변환
dev_vectors = vectorizer.transform(dev_data['text'])        # 검증 데이터를 숫자로 변환
test_vectors = vectorizer.transform(test_data['text'])      # 테스트 데이터를 숫자로 변환

# print(train_vectors)      # transform 된 결과 확인


##### 텍스트와 범주형 특성 다루기 (Bag-of-words 방식으로 벡터화 한 것 확인하는 방법 구현)
sample_num = -1                            # 확인하고 싶은 샘플 번호
sample_origin = train_data[sample_num]    # 확인하고 싶은 샘플의 원래 문장
sample_transform = train_vectors[sample_num]    # 확인하고 싶은 샘플의 transform된 결과
sample_inverse_transform = vectorizer.inverse_transform(sample_transform) # 확인하고 싶은 샘플의 transform된 결과를 다시 단어의 조합으로 바꾼 결과
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))    # 출력

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Original Input:{'label': 0, 'text': "큰 의미 없이 연결 ' 만 ' 되 는 살인 들 ."}
Transformed:   (0, 203471)	0.3094354542114961
  (0, 203417)	0.20712451996101208
  (0, 162708)	0.3094354542114961
  (0, 162691)	0.19666660829166394
  (0, 140038)	0.3306610010321105
  (0, 140033)	0.2498478108712802
  (0, 133337)	0.3306610010321105
  (0, 133216)	0.1743997187719626
  (0, 106512)	0.3306610010321105
  (0, 106500)	0.23398736211217852
  (0, 79732)	0.28820990739088176
  (0, 79531)	0.11015822978377737
  (0, 66178)	0.09557080561241525
  (0, 63733)	0.1604998613699301
  (0, 63698)	0.1253209800929586
  (0, 45667)	0.31824485208474995
  (0, 44295)	0.06513681141714016
Inv-transformed: [array(['큰 의미', '큰', '의미 없이', '의미', '연결 만', '연결',
       '없이 연결', '없이', '살인 들', '살인', '만 되', '만',
       '들', '되 는', '되', '는 살인', '는'], dtype='<U69')]


In [74]:
##### 훈련세트에서 훈련하고 평가하기
from sklearn.svm import LinearSVC     #Linear kernel SVM을 사용하기 위한 클래스     

svm = LinearSVC(random_state = False)     # 모델 정의
svm.fit(train_vectors, train_data['label'])     # 모델 학습

print(svm.coef_)      # weights (w)
print(svm.intercept_)     # bias (b)

[[-0.19180475 -0.19180475  0.         ...  0.15471785 -0.00546948
  -0.00546948]]
[-0.13440725]


In [75]:
##### 교차 검증을 사용한 평가

from sklearn.model_selection import cross_val_score

all_data = train_data['text']+dev_data['text']+test_data['text']      # all_data = train_data + dev_data + test_data
all_label = train_data['label']+dev_data['label']+test_data['label']  # all_label은 train_data, dev_data, test_date의 레이블을 모두 합한 것 
all_vectors = vectorizer.transform(all_data)    # all_data를 숫자로 변환
scores = cross_val_score(svm, all_vectors, all_label, cv=5)  # 5-cross validation

print("All scores:", scores)  # 전체 점수 출력
print("Average: {:.2f}%".format(scores.mean()*100)) # 다섯 번의 스코어 평균 출력
print("Standard deviation: {:.6f}".format(scores.std()))  # 표준편차 출력
 

All scores: [0.8445   0.84925  0.847625 0.843    0.854   ]
Average: 84.77%
Standard deviation: 0.003858


In [76]:
##### 그리드 탐색
# from sklearn.model_selection import GridSearchCV    

# param_grid = [{'max_iter':[500, 1000, 5000], 'C': [1, 10, 100]}]        # 학습 max_iter와 C를 변동 하이퍼파라미터로 설정
# grid_search = GridSearchCV(svm, param_grid, cv=3)     # 하이퍼파라미터 서치 할 모델 설정, 각 모델별로 5-cross validation 실행
# grid_search.fit(train_vectors, train_data['label'])   # transformed train data로 학습

# print(grid_search.cv_results_['mean_test_score'])     # 파라미터 서치 결과
# print(grid_search.best_params_)   #최고의 모델 파라미터 

# #파라미터 서치 결과 출력
# for mean_score, params in zip(grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['params']):
#   print(mean_score, params)       

In [77]:
##### 테스트 세트로 시스템 평가하기
from sklearn.metrics import accuracy_score    # Accuracy 측정 함수 import

final_model = svm #grid_search.best_estimator_     # grid search에 의해 정해진 제일 좋은 하이퍼파라미터

pred_results = final_model.predict(test_vectors)    # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results)   # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100))   # 정확도 출력 

Accuracy: 85.98%
